In [4]:
from data_fetcher import DataFetcher

DataFetcher(1000000, True, True)

Will try to save to /workspaces/pagerank/src/../data/wikilinks.csv
Sending request to "https://downloads.dbpedia.org/repo/dbpedia/generic/wikilinks/2022.12.01/wikilinks_lang=en.ttl.bz2" ...
Starting decompressing data from the original file
Only some of triples were extracted...
Output of size 1.000088 MB.


In [5]:
import os
from operator import add
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, lit
from pyspark.sql.types import StructType, StructField, StringType

NUMBER_ITERATION = 10

In [6]:
def compute_contrib(children, rank):
    number_children = len(children)
    for c in children:
        yield c, rank / number_children

In [7]:
def csv_reader(s):
    elements = s.split(';')
    return (elements[0].strip('"'), elements[1].strip('"').split(' '))

In [10]:
def PageRank_RDD(nombre_iteration:int, data_path:str="../data/small_sample.csv"):
    # Creation de l'application
    spark = SparkSession.builder.appName("pagerank_rdd").getOrCreate()
    sc = spark.sparkContext
    
    data = sc.textFile(data_path).map(csv_reader)
    data = data.partitionBy(sc.defaultParallelism).persist()
    print('Parallelism:', sc.defaultParallelism)

    # Initial ranks: 1.0 pour chaque node présent dans adj keys ou values
    link_src = data.keys()
    link_dst = data.flatMap(lambda row: row[1])
    ranks = link_src.union(link_dst).distinct().map(lambda url:(url,1.0))

    # Partitionnement
    ranks = ranks.partitionBy(sc.defaultParallelism).persist()
    ranks.count()

    # Calcul du page rank
    for _ in range(nombre_iteration):
        ranks = data.join(ranks)\
            .flatMap(lambda row: compute_contrib(row[1][0], row[1][1]))\
            .reduceByKey(add)\
            .mapValues(lambda rank : rank * 0.85 + 0.15)

    return ranks.sortBy(lambda elm: elm[1], ascending=False)

In [12]:
PageRank_RDD(NUMBER_ITERATION, '../data/wikilinks.csv').collect()

Parallelism: 2


25/12/02 12:38:40 ERROR Executor: Exception in task 0.0 in stage 66.0 (TID 70)2]
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/usr/local/python/3.12.1/lib/python3.12/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 2044, in main
    process()
  File "/usr/local/python/3.12.1/lib/python3.12/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 2036, in process
    serializer.dump_stream(out_iter, outfile)
  File "/usr/local/python/3.12.1/lib/python3.12/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 273, in dump_stream
    vs = list(itertools.islice(iterator, batch))
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/python/3.12.1/lib/python3.12/site-packages/pyspark/python/lib/pyspark.zip/pyspark/util.py", line 129, in wrapper
    def wrapper(*args: Any, **kwargs: Any) -> Any:
    
KeyboardInterrupt

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.

KeyboardInterrupt: 